In [3]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import igl
import meshplot as mp
import json
from PIL import Image
from pylab import imshow, plot

In [62]:
mesh_file = 'dataset/RGB3DCaricShop_raw_t_mesh_eye_plus/processedData/tMesh/Arnold Schwarzenegger/C00015.obj'
img_file = 'dataset/RGB3DCaricShop_raw_t_mesh_eye_plus/processedData/img/Arnold Schwarzenegger/C00015.jpg'
cam_file = 'dataset/RGB3DCaricShop_raw_t_mesh_eye_plus/processedData/calib/Arnold Schwarzenegger/C00015.mat'

In [63]:
v, f = igl.read_triangle_mesh(mesh_file)
img = Image.open(img_file)
np_img = np.array(img)
cam = sio.loadmat(cam_file)

In [64]:
cam_ext = cam['Ext']
cam_int = cam['Int']
cam_int = np.column_stack([np.row_stack([cam_int, np.array([0., 0., 1.])]), np.zeros([3])])
cam = cam_int @ cam_ext

In [65]:
cam

array([[-6.83901093e+02,  2.69318281e+00,  3.20693087e+02,
        -7.98088357e+02],
       [ 5.09691319e+01,  6.88155054e+02,  3.07273642e+02,
        -7.65952428e+02],
       [ 9.09095000e-02, -7.80262000e-02,  9.92798000e-01,
        -3.06583609e+00]])

In [66]:
cam.shape

(3, 4)

In [67]:
v.shape

(35200, 3)

In [68]:
wv = np.row_stack([v.T, np.ones([len(v)])])

In [69]:
wv.shape

(4, 35200)

In [70]:
project_v = cam @ wv

In [71]:
project_v.shape

(3, 35200)

In [72]:
v_uv = project_v[:2] / project_v[2:]

In [51]:
def render_image(vertices, faces, ax):
    triangles = [vertices[face] for face in faces]
    ax.add_collection(PolyCollection(triangles, edgecolor='k', facecolor='gray', closed=True))

In [22]:
from matplotlib.collections import PolyCollection

In [73]:
v_uv.shape

(2, 35200)

In [55]:
from PIL import Image
import numpy as np

def sample_image(image_path, points):
    # 打开原始图片
    original_image = Image.open(image_path)

    # 获取图片的宽度和高度
    width, height = original_image.size

    # 创建一个新的空白图片，初始为全黑
    new_image = Image.new('RGB', (width, height), color='black')

    # 将原始图片的 RGB 值根据给定的点坐标进行抽样，并填充到新图片中
    for point in points:
        x, y = point
        # 确保点在图片范围内
        if 0 <= x < width and 0 <= y < height:
            # 获取原始图片上该点的 RGB 值
            rgb = original_image.getpixel((x, y))
            # 在新图片上填充该点的 RGB 值
            new_image.putpixel((x, y), rgb)

    # 保存新图片
    new_image.save('output_image.jpg')

    

In [76]:
from staticdata.rawlandmarks import CaricShop3D

In [90]:
idx = list(CaricShop3D.MAP_ORIG_TO_SLICED)

In [86]:
max(CaricShop3D.MAP_ORIG_TO_SLICED)

10410

In [94]:
len(idx)

9725

In [93]:
idx.extend(range(11551, 11739))

In [78]:
v_uv.T.shape

(35200, 2)

In [95]:
image_path = img_file

# 替换这里的points变量为你的点集合
points = np.round(v_uv.T[idx]).astype(int)

sample_image(image_path, points)